In [ ]:
import numpy as np
import torch

from visualization import *
from main import *
from used_attacks import regression_PGD

""" Settings and seeds """

# Set default dtype to float32
torch.set_default_dtype(torch.float)

# PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("mps")

print("device:", device)

if device == "cuda":
    print(torch.cuda.get_device_name())

""" test dataset """


def poisson2d_gt(x, y):
    return (
        np.exp(-np.square(10 * x - 8))
        - np.exp(-np.square(10 * y - 8))
        + np.exp(-np.square(10 * x + 8))
        - np.exp(-np.square(10 * y + 8))
        + np.exp(-np.square(10 * x))
        - np.exp(-np.square(10 * y))
    )


x = np.linspace(-1, 1, 256)
y = np.linspace(-1, 1, 256)
X, Y = np.meshgrid(x, y)
U_gt = poisson2d_gt(X, Y)

x_test = np.hstack((X.flatten()[:, None], Y.flatten()[:, None]))
u_test = U_gt.flatten()[:, None]
print("test dataset:", x_test.shape, "\n")

""" model """

layers = np.array([2, 20, 20, 20, 20, 20, 20, 20, 20, 1])  # 8 hidden layers
PINN = sequential_model(layers, device)
PINN.to(device)
print(PINN)
plot_u(U_gt, x, y, title="$u(x)$")

In [ ]:
def generate_attack_samples(
    model,
    device,
    x_train,
    N0,
    n_samples=1000,
    lb=[-1, -1],
    ub=[1, 1],
    steps=20,
    eps=1e-1,
    eta=2e-2,
    m=1,
):
    attack = regression_PGD(model, lb=lb, ub=ub, steps=steps, eps=eps, eta=eta)
    x_adv = attack.attack(x_train).cpu().detach().numpy()

    if x_adv.shape[0] > N0 + (m + 1) * n_samples:
        x_adv = np.vstack([x_adv[:N0, :], x_adv[-(m + 1) * n_samples :, :]])

    f_adv_pred = np.abs(model.function(x_adv).squeeze().cpu().detach().numpy())
    index = np.argsort(f_adv_pred)[::-1][:n_samples]
    x_adv = x_adv[index, :]
    f_adv_pred = f_adv_pred[index]

    plot_samples(x_adv, f_adv_pred)

    return x_adv


def retrain(
    model, x_train, x_adv, x_boundary, u_boundary, x_test, u_test, U_gt, epoch=1e5
):
    f_test_pred = np.abs(PINN.function(x_test).squeeze().cpu().detach().numpy())
    plot_u(f_test_pred.reshape(X.shape).T, x, y, log=True, title="test $r(x;theta)$")

    model.iter = 0
    optimizer = torch.optim.Adam(PINN.parameters(), lr=0.0001)
    model.train_model_adam(
        optimizer, x_boundary, u_boundary, np.vstack([x_train, x_adv]), epoch
    )

    rmse, u_pred = model.test(x_test, u_test)
    print("Test RMSE: %.5f" % (rmse))

    plot_u_x(u_pred.reshape(X.shape).T, U_gt, x, y)
    plot_u(
        np.abs(u_pred.reshape(X.shape).T - U_gt),
        x,
        y,
        log=True,
        title="$|u(x;theta)-u(x)|$",
    )

In [ ]:
""" k=0 """
N0 = 500
x_train, x_boundary, u_boundary = training_data_latin_hypercube(X, Y, U_gt, N_inner=N0)
retrain(
    PINN,
    x_train,
    np.array([]).reshape((0, 2)),
    x_boundary,
    u_boundary,
    x_test,
    u_test,
    U_gt,
    epoch=2e5,
)

In [ ]:
""" k=1 """
x_adv = generate_attack_samples(PINN, device, x_train, N0, n_samples=500)
retrain(PINN, x_train, x_adv, x_boundary, u_boundary, x_test, u_test, U_gt, epoch=5e4)

In [ ]:
""" k=2:12 """
for i in range(11):
    x_adv = generate_attack_samples(PINN, device, x_train, N0, n_samples=1000)
    retrain(
        PINN, x_train, x_adv, x_boundary, u_boundary, x_test, u_test, U_gt, epoch=5e4
    )